<a href="https://colab.research.google.com/github/Random-Coder-Dude/Realtime-Object-Detection/blob/main/limelight-detector-training-notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![TrainingNotebookLogo.png](https://downloads.limelightvision.io/content/TrainingNotebookLogo.png)

To train a neural object detector for Limelight, click the "play" button on each code block. Pay extra attention to any "❗" you see. By the end of this tutorial, you will have downloaded a .zip file containing your model and label files.

See https://docs.limelightvision.io/docs/docs-limelight/pipeline-neural/training-your-own-detector for a more in-depth tutorial.

# 1. Install The Object Detection Package

In [15]:
import shutil
import os
tmpModelPath ='/content/models'
if os.path.exists(tmpModelPath) and os.path.isdir(tmpModelPath):
  shutil.rmtree(tmpModelPath)

MLENVIRONMENT="COLAB"
!git clone --depth 1 https://github.com/tensorflow/models
!cd models && git fetch --depth 1 origin ad1f7b56943998864db8f5db0706950e93bb7d81 && git checkout ad1f7b56943998864db8f5db0706950e93bb7d81
!pip install protobuf==3.20.3

fatal: destination path 'models' already exists and is not an empty directory.
remote: Enumerating objects: 3055, done.
remote: Counting objects: 100% (3055/3055), done.
remote: Compressing objects: 100% (1359/1359), done.
remote: Total 1824 (delta 1223), reused 703 (delta 446), pack-reused 0 (from 0)
Receiving objects: 100% (1824/1824), 10.05 MiB | 7.01 MiB/s, done.
Resolving deltas: 100% (1223/1223), completed with 739 local objects.
From https://github.com/tensorflow/models
 * branch            ad1f7b56943998864db8f5db0706950e93bb7d81 -> FETCH_HEAD
HEAD is now at ad1f7b5 adjust folder path
Defaulting to user installation because normal site-packages is not writeable


In [16]:
# Environment Setup
import os
import sys
import re

print(sys.version)
if(MLENVIRONMENT == "COLAB"):
    print("colab env setup")
    os.environ["HOMEFOLDER"] = "/home/atharv/"
    HOMEFOLDER = '{HOMEFOLDER}'.format(**os.environ)
    FINALOUTPUTFOLDER_DIRNAME = 'final_output'
    FINALOUTPUTFOLDER = HOMEFOLDER+FINALOUTPUTFOLDER_DIRNAME
    print(HOMEFOLDER)

# Copy setup files into models/research folder
!cd {HOMEFOLDER}models/research && pwd && protoc object_detection/protos/*.proto --python_out=.

# Modify setup.py
with open(HOMEFOLDER+'models/research/object_detection/packages/tf2/setup.py') as f:
    s = f.read()

with open(HOMEFOLDER+'models/research/setup.py', 'w') as f:
    if(MLENVIRONMENT == "COLAB"):
        s = re.sub('tf-models-official>=2.5.1','tf-models-official==2.15.0', s)
        f.write(s)

3.10.12 (main, Jan 17 2025, 14:35:34) [GCC 11.4.0]
colab env setup
/home/atharv/
/home/atharv/models/research


In [17]:
# Install
!pip install {HOMEFOLDER}models/research/
if(MLENVIRONMENT == "COLAB"):
    !pip install tensorflow==2.15.0
    !pip install protobuf==3.20.3

Defaulting to user installation because normal site-packages is not writeable
Processing ./models/research
  Preparing metadata (setup.py) ... - done
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1697752 sha256=d923fdeacc69c8d4bbbf250eb82a4d10345a6d0294cd441cd7a118112ace7121
  Stored in directory: /tmp/pip-ephem-wheel-cache-hhhrirws/wheels/d2/91/19/8fcd2d3d66b31c32d80d2d33f433eeb698c65e1c289e384aa1
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Test the environment by running `model_builder_tf2_test.py` to make sure everything is working as expected.

In [18]:
!python3 {HOMEFOLDER}models/research/object_detection/builders/model_builder_tf2_test.py

2025-02-08 17:35:23.667684: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-08 17:35:23.667716: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-08 17:35:23.668566: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-08 17:35:23.672823: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-08 17:35:24.219673: W tensorflow/compiler/tf2

# 1.1. Get Dataset From Google Drive

1. Expand this section
2. Upload your RoboFlow .tfrecord.zip to Google Drive
3. Share the uploaded .tfrecord.zip such that anyone with the link can access the file.
4. Run this block
5. Paste your Google Drive file share link into the text box that appears after running this block
6. Click the "Process Dataset" Buttton
7. Click the Refresh button in the "Files" pane to ensure dataset.zip exists

In [ ]:
import gdown
import os
from IPython.display import display
from ipywidgets import Text, Button, VBox

def process_dataset():
    link_input = Text(
        value='',
        placeholder='Paste your Google Drive share link here',
        description='Drive Link:',
        style={'description_width': 'initial'},
        layout={'width': '50%'}
    )

    def on_click(b):
        try:
            print("Downloading dataset...")
            url = link_input.value

            # Convert share URL to direct download URL if needed
            if 'drive.google.com/file/d/' in url:
                file_id = url.split('/file/d/')[1].split('/')[0]
                url = f'https://drive.google.com/uc?id={file_id}'

            output = '/content/dataset.zip'
            gdown.download(url, output, fuzzy=True)
            print("Download complete!")

        except Exception as e:
            print(f"Error: {str(e)}")

    process_button = Button(description='Process Dataset', button_style='primary')
    process_button.on_click(on_click)
    display(VBox([link_input, process_button]))

# Install gdown if not already installed
!pip install -q gdown --upgrade

# Execute
process_dataset()

# 2. Auto-detect relevant tfrecord components

In [5]:
datasetPath = '/home/atharv/dataset.zip'
print(datasetPath)
!unzip $datasetPath

/home/atharv/dataset.zip
Archive:  /home/atharv/dataset.zip
replace README.dataset.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [10]:
import os
import fnmatch

def find_files(directory, pattern):
    for root, dirs, files in os.walk(directory):
        for basename in files:
            if fnmatch.fnmatch(basename, pattern):
                filename = os.path.join(root, basename)
                yield filename

def set_tfrecord_variables(directory):
    train_record_fname = ''
    val_record_fname = ''
    label_map_pbtxt_fname = ''

    for tfrecord_file in find_files(directory, '*.tfrecord'):
        if '/train/' in tfrecord_file:
            train_record_fname = tfrecord_file
        elif '/valid/' in tfrecord_file:
            val_record_fname = tfrecord_file
        elif '/test/' in tfrecord_file:
            pass

    for label_map_file in find_files(directory, '*_label_map.pbtxt'):
        label_map_pbtxt_fname = label_map_file  # Assuming one common label map file

    return train_record_fname, val_record_fname, label_map_pbtxt_fname


train_record_fname, val_record_fname, label_map_pbtxt_fname = set_tfrecord_variables('/home/atharv')

train_record_fname = '/home/atharv/train/coral-and-algae.tfrecord'
val_record_fname = '/home/atharv/valid/coral-and-algae.tfrecord'
label_map_pbtxt_fname = '/home/atharv/train/coral-and-algae_label_map.pbtxt'

#if(MLENVIRONMENT=="COLAB"):
    #train_record_fname = '/content/train/cubes-cones.tfrecord'
    #val_record_fname = '/content/valid/cubes-cones.tfrecord'
    #label_map_pbtxt_fname = '/content/train/cubes-cones_label_map.pbtxt'

print("Train Record File:", train_record_fname)
print("Validation Record File:", val_record_fname)
print("Label Map File:", label_map_pbtxt_fname)



Train Record File: /home/atharv/train/coral-and-algae.tfrecord
Validation Record File: /home/atharv/valid/coral-and-algae.tfrecord
Label Map File: /home/atharv/train/coral-and-algae_label_map.pbtxt


# 3.&nbsp;Training Configuration and Labels File Generation

Download the pre-trained Limelight Base Model

In [11]:
chosen_model = 'ssd-mobilenet-v2'
MODELS_CONFIG = {
    'ssd-mobilenet-v2': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'limelight_ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'limelight_ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
    },
}
model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

# Create "mymodel" folder for pre-trained weights and configuration files
%cd ~
%mkdir {HOMEFOLDER}models/mymodel/
%cd {HOMEFOLDER}models/mymodel/
%pwd

# Download pre-trained model weights
import tarfile
download_tar = 'https://downloads.limelightvision.io/models/' + pretrained_checkpoint
!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

# Download training configuration file for model
download_config = 'https://downloads.limelightvision.io/models/' + base_pipeline_file
!wget {download_config}
%cd ~

# Set training parameters for the model
num_steps = 40000
checkpoint_every = 2000
batch_size = 16


/home/atharv
mkdir: cannot create directory ‘/home/atharv/models/mymodel/’: File exists
/home/atharv/models/mymodel
--2025-02-08 17:33:22--  https://downloads.limelightvision.io/models/limelight_ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
Resolving downloads.limelightvision.io (downloads.limelightvision.io)... 

/home/atharv/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/atharv/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


2600:9000:20ed:e800:c:efbc:d340:93a1, 2600:9000:20ed:7200:c:efbc:d340:93a1, 2600:9000:20ed:600:c:efbc:d340:93a1, ...
Connecting to downloads.limelightvision.io (downloads.limelightvision.io)|2600:9000:20ed:e800:c:efbc:d340:93a1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46042990 (44M) [application/x-gzip]
Saving to: ‘limelight_ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz.2’

limelight_ssd_mobil 100%[===================>]  43.91M  5.92MB/s    in 8.5s    

2025-02-08 17:33:31 (5.14 MB/s) - ‘limelight_ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz.2’ saved [46042990/46042990]

--2025-02-08 17:33:32--  https://downloads.limelightvision.io/models/limelight_ssd_mobilenet_v2_320x320_coco17_tpu-8.config
Resolving downloads.limelightvision.io (downloads.limelightvision.io)... 2600:9000:20ed:7c00:c:efbc:d340:93a1, 2600:9000:20ed:e800:c:efbc:d340:93a1, 2600:9000:20ed:7200:c:efbc:d340:93a1, ...
Connecting to downloads.limelightvision.io (downloads.limelightvision.io)|2

Generate Labels File

In [12]:

# Set file locations and get number of classes for config file
pipeline_fname = HOMEFOLDER+'models/mymodel/' + base_pipeline_file
fine_tune_checkpoint = HOMEFOLDER+'models/mymodel/' + model_name + '/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

def get_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)

    class_names = [category['name'] for category in category_index.values()]
    return class_names

def create_label_file(filename, labels):
    with open(filename, 'w') as file:
        for label in labels:
            file.write(label + '\n')


num_classes = get_num_classes(label_map_pbtxt_fname)
classes = get_classes(label_map_pbtxt_fname)

print('Total classes:', num_classes)
print(classes)


#Generate labels file
create_label_file(HOMEFOLDER + "limelight_neural_detector_labels.txt", classes)

2025-02-08 17:33:36.559636: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-08 17:33:36.559664: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-08 17:33:36.560514: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-08 17:33:36.565262: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-08 17:33:37.242990: W tensorflow/compiler/tf2

Total classes: 2
['ALGAE', 'CORAL']


Modify the base Limelight Model Configuration File

Augmentation Options: https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto

In [13]:
# Create custom configuration file by writing the dataset, model checkpoint, and training parameters into the base pipeline file
import re

print('writing custom configuration file')



with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:

    # Set fine_tune_checkpoint path
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)

    # Set tfrecord files for train and test datasets
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(val_record_fname), s)

    # Set label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set batch_size
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)

    # Set number of classes num_classes
    s = re.sub('checkpoint_every_n: [0-9]+',
               'num_classes: {}'.format(num_classes), s)

    # Change fine-tune checkpoint type from "classification" to "detection"
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)

    # If using ssd-mobilenet-v2, reduce learning rate
    if chosen_model == 'ssd-mobilenet-v2':
      s = re.sub('learning_rate_base: .8',
                 'learning_rate_base: .004', s)

      s = re.sub('warmup_learning_rate: 0.13333',
                 'warmup_learning_rate: .0016666', s)

    # If using efficientdet-d0, use fixed_shape_resizer instead of keep_aspect_ratio_resizer (because it isn't supported by TFLite)
    if chosen_model == 'efficientdet-d0':
      s = re.sub('keep_aspect_ratio_resizer', 'fixed_shape_resizer', s)
      s = re.sub('pad_to_max_dimension: true', '', s)
      s = re.sub('min_dimension', 'height', s)
      s = re.sub('max_dimension', 'width', s)

    f.write(s)

# (Optional) Display the custom configuration file's contents
# !cat pipeline_file.config
# Set the path to the custom config file and the directory to store training checkpoints in
pipeline_file = 'pipeline_file.config'
model_dir = HOMEFOLDER+'training_progress/'
print(" ")
print(model_dir)

writing custom configuration file
 
/home/atharv/training_progress/


# 4.&nbsp;Train Model

Once training starts, come back and click the refresh button within the tensorboard window to check training progress.



In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/training_progress/train'

Fix TF 2.15 breaking changes

In [19]:
import shutil
import re

original_path = '/home/atharv/.local/lib/python3.10/site-packages/tf_slim/data/tfexample_decoder.py'
with open(original_path, 'r') as file:
  content = file.read()
  content = re.sub(r'import abc', 'import tensorflow as tf\n\nimport abc', content)
  content = re.sub(r'control_flow_ops.case', 'tf.case', content)
  content = re.sub(r'control_flow_ops.cond', 'tf.compat.v1.cond', content)
with open(original_path, 'w') as file:
  file.write(content)

print(f"File {original_path} fixed.")

File /home/atharv/.local/lib/python3.10/site-packages/tf_slim/data/tfexample_decoder.py fixed.


Train

In [20]:
!rm -rf {HOMEFOLDER}training_progress
# Run training!
!python3 {HOMEFOLDER}models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --checkpoint_every_n={checkpoint_every} \
    --num_train_steps={num_steps} \
    --num_workers=2 \
    --sample_1_of_n_eval_examples=1

2025-02-08 17:36:02.007450: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-08 17:36:02.007483: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-08 17:36:02.008388: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-08 17:36:02.012604: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-08 17:36:02.593053: W tensorflow/compiler/tf2

Feel free to stop training early. Check the 'training_progress' folder to see all training checkpoints.


# 5.&nbsp;Convert Model to TFLite

In [ ]:
#remove final output folder if it exists
if os.path.exists(FINALOUTPUTFOLDER) and os.path.isdir(FINALOUTPUTFOLDER):
  shutil.rmtree(FINALOUTPUTFOLDER)

# Make a directory to store the trained TFLite model
!mkdir {FINALOUTPUTFOLDER}
print(FINALOUTPUTFOLDER)
# Export graph
# Path to training directory (the conversion script automatically chooses the highest checkpoint file)
last_model_path = HOMEFOLDER+'training_progress'
exporter_path = HOMEFOLDER+'models/research/object_detection/export_tflite_graph_tf2.py'
output_directory = FINALOUTPUTFOLDER

!python $exporter_path \
    --trained_checkpoint_dir $last_model_path \
    --output_directory $output_directory \
    --pipeline_config_path $pipeline_file

# Convert to .tflite Flatbuffer
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model(FINALOUTPUTFOLDER+'/saved_model')
tflite_model = converter.convert()
model_path_32bit = FINALOUTPUTFOLDER+'/limelight_neural_detector_32bit.tflite'
with open(model_path_32bit, 'wb') as f:
  f.write(tflite_model)

!cp {HOMEFOLDER}limelight_neural_detector_labels.txt {FINALOUTPUTFOLDER}
!cp {HOMEFOLDER}models/mymodel/pipeline_file.config {FINALOUTPUTFOLDER}

In [ ]:
# Export graph
# Path to training directory (the conversion script automatically chooses the highest checkpoint file)
last_model_path = HOMEFOLDER+'training_progress'
exporter_path = HOMEFOLDER+'models/research/object_detection/export_tflite_graph_tf2.py'
output_directory = FINALOUTPUTFOLDER

!python $exporter_path \
    --trained_checkpoint_dir $last_model_path \
    --output_directory $output_directory \
    --pipeline_config_path $pipeline_file

# Convert to .tflite Flatbuffer
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model(FINALOUTPUTFOLDER+'/saved_model')
tflite_model = converter.convert()
model_path_32bit = FINALOUTPUTFOLDER+'/limelight_neural_detector_32bit.tflite'
with open(model_path_32bit, 'wb') as f:
  f.write(tflite_model)

!cp {HOMEFOLDER}limelight_neural_detector_labels.txt {FINALOUTPUTFOLDER}

# 6. Quantize model
The "TFLiteConverter" module will perform [post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization) on the model. To quantize the model, we need to provide a set of example images. We will extract 100 images from the training tfrecord and place said images into the "extracted_samples" folder.


In [ ]:
import tensorflow as tf
import os
import io
from PIL import Image

def extract_images_from_tfrecord(tfrecord_path, output_folder, num_samples=100):
    # Make sure the output directory exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Initialize a counter for the number of images saved
    saved_images = 0

    # Read the TFRecord file
    raw_dataset = tf.data.TFRecordDataset(tfrecord_path)
    for raw_record in raw_dataset.take(num_samples):
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())

        # Extract the image data (change 'image/encoded' if necessary)
        image_data = example.features.feature['image/encoded'].bytes_list.value[0]

        # Decode the image data and save as a file
        image = Image.open(io.BytesIO(image_data))
        image.save(os.path.join(output_folder, f'image_{saved_images}.png'))

        saved_images += 1
        if saved_images >= num_samples:
            break

    print(f"Extracted {saved_images} images to {output_folder}")

# Set the path to your TFRecord file and the output directory
tfrecord_path = train_record_fname
extracted_sample_folder = HOMEFOLDER+'extracted_samples'

#remove sample folder if it exists
if os.path.exists(extracted_sample_folder) and os.path.isdir(extracted_sample_folder):
  shutil.rmtree(extracted_sample_folder)

# Extract images
extract_images_from_tfrecord(tfrecord_path, extracted_sample_folder)


# Get list of all images in train directory
from google.cloud import storage
import glob

quant_image_list=[]
if(MLENVIRONMENT=="COLAB"):

    jpg_file_list = glob.glob(extracted_sample_folder + '/*.jpg')
    jpeg_file_list = glob.glob(extracted_sample_folder + '/*.jpeg')
    JPG_file_list = glob.glob(extracted_sample_folder + '/*.JPG')
    png_file_list = glob.glob(extracted_sample_folder + '/*.png')
    bmp_file_list = glob.glob(extracted_sample_folder + '/*.bmp')
    quant_image_list = jpg_file_list + JPG_file_list + png_file_list + bmp_file_list

print("pulling samples from " + extracted_sample_folder)
print("samples: " + str(len(quant_image_list)))

In [ ]:
# A generator that provides a representative dataset
# Code modified from https://colab.research.google.com/github/google-coral/tutorials/blob/master/retrain_classification_ptq_tf2.ipynb

# First, get input details for model so we know how to preprocess images
interpreter = tf.lite.Interpreter(model_path=model_path_32bit)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]

import random

def representative_data_gen():
  dataset_list = quant_image_list
  quant_num = 300
  for i in range(quant_num):
    pick_me = random.choice(dataset_list)
    print(pick_me)
    image = tf.io.read_file(pick_me)

    if pick_me.endswith('.jpg') or pick_me.endswith('.JPG') or pick_me.endswith('.jpeg'):
      image = tf.io.decode_jpeg(image, channels=3)
    elif pick_me.endswith('.png'):
      image = tf.io.decode_png(image, channels=3)
    elif pick_me.endswith('.bmp'):
      image = tf.io.decode_bmp(image, channels=3)

    image = tf.image.resize(image, [width, height])  # TO DO: Replace 300s with an automatic way of reading network input size
    image = tf.cast(image / 255., tf.float32)
    image = tf.expand_dims(image, 0)
    yield [image]

Finally, we'll initialize the TFLiteConverter module, point it at the TFLite graph we generated in Step 6, and provide it with the representative dataset generator function we created in the previous code block. We'll configure the converter to quantize the model's weight values to INT8 format.

In [ ]:
# Initialize converter module
converter = tf.lite.TFLiteConverter.from_saved_model(FINALOUTPUTFOLDER+'/saved_model')
print("initialized converter")
# This enables quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# This sets the representative dataset for quantization
converter.representative_dataset = representative_data_gen
# This ensures that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# For full integer quantization, though supported types defaults to int8 only, we explicitly declare it for clarity.
converter.target_spec.supported_types = [tf.int8]
# These set the input tensors to uint8 and output tensors to float32
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.float32
print("begin conversion")
tflite_model = converter.convert()
print("conversion complete")

with open(FINALOUTPUTFOLDER+'/limelight_neural_detector_8bit.tflite', 'wb') as f:
  f.write(tflite_model)

# 7. Compile Model for Limelight & Download


Install Coral Compiler

In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
! sudo apt-get update
! sudo apt-get install edgetpu-compiler

Compile the previously-generated 8-bit model for Google Coral

In [ ]:
!cd {FINALOUTPUTFOLDER} && pwd && edgetpu_compiler limelight_neural_detector_8bit.tflite && pwd && mv limelight_neural_detector_8bit_edgetpu.tflite limelight_neural_detector_coral.tflite && rm limelight_neural_detector_8bit_edgetpu.log

Zip models

In [ ]:
!rm {HOMEFOLDER}limelight_detectors.zip
!zip -r {HOMEFOLDER}limelight_detectors.zip {FINALOUTPUTFOLDER}

Download

In [ ]:
from google.colab import files
files.download(HOMEFOLDER+'limelight_detectors.zip')